## Gradienter, backpropagation

Gradienterna i djupa nätverk är inte stabila vid numerisk derivering (beräkning av gradienterna). Två huvudsakliga beteenden observeras:
* Försvinnande gradient (Vanishing Gradient) -- gradienten går mot noll desto djupare i närverket algoritmen går. Alltså slutar nätverket lära sig och de djupa lagren slutar uppdateras. Nätverket konvergerar inte mot någon lösning.
* Exploderande gradient (Exploding Gradient) -- gradienten divergerar och därmed även den förutsagda lösningen. Dessa gradienter kan till och med vara periodiska eller kaotiska. Förekommer i allmänhet bara i _rekurrenta_ nätverk (RNN), även om undantag finns!

### Vad som händer matematiskt

Detta var, som så mycket annat, en empirisk upptäckt. Ingen matematik fanns som förklarade beteendet
och ledde till att DNN (djupa neurala nätverk) övergavs kring 2000. Xavier Glorot och Yoshua Bengio 
visade kring 2010 att kombinationen av den aktiveringsfunktion som var mest populär (sigmoid) och 
hur vikterna i nätverket initialiserades ledde till att variansen ökade i varje lager. Det var alltså
en kombination av en statistisk effekt och ett numerisk närmevärde.

<img src="../Data/sigmoid_sat.jpg" />



I allmänhet kan inte variansen för både gradienterna och grundsanningen hållas nere om inte alla lager har samma antal kopplingar in och ut, vilket är en ytterligare anledning varför vissa nätverk föredrar att hålla samma antal noder i alla lager. Men Glorot och Bengio hittade en bra kompromiss, som visar sig fungera bra nog i praktiken: så-kallad _Glorot initialisering_.

Men den tekniken gäller bara för sigmoida funktioner. För ReLU föredras _He-initialisering_ (även _Kaiming-initialisering_ efter forskaren Kaiming He). Dessa skiljer sig mest i skalfaktorer och vilken statistika som används för variansen (in, ut eller medel). En till vanlig initalisering är _LeCun_ initialisering, som oftast används med SELU men även just nätverk där antalet noder i alla lager är detsamma.

<img src="../Data/active_init.jpg">

In [ ]:
import torch
import torch.nn as nn
# av historiska skäl fungerar initialisering lite konstigt i Torch
# bäst är att göra det explicit:

layer = nn.Linear(40,10)

## He-initialisation
nn.init.kaiming_uniform_(layer.weight)
nn.init.zeros_(layer.bias) # bias kan lika gärna börja på noll

Parameter containing:
tensor([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], requires_grad=True)

### Aktiveringsfunktioner

Så vilken aktiveringsfunktion skall man välja?

I praktiken är ReLU väldigt snabb att beräkna och mättas inte för positiva värden (har nollskild gradient), men den har ett problem som kallas "döende ReLU" -- under träningen kan dessa aktiveringsfunktioner hamna i ett läga då de alltid får negativ input och alltså alltid ger tillbaka 0. Därefter är den noden permanent 0, och alltså "död". 

En lösning är LeakyReLU:
<img src="../Data/LeakReLU.jpg">

Även en variant där vinkeln är en parameter, PReLU, finns.

In [ ]:
alpha = 0.2
model = nn.Sequential(
    nn.Linear(50, 40),
    nn.LeakyReLU(negative_slope=alpha)    
)
nn.init.kaiming_uniform_(model[0].weight, alpha, nonlinearity="leaky_relu")

#### GELU, Swish, SwiGLU, Mish, RELU²

<img src="../Data/GELU.jpg">

GELU, Mish och Swish finns som <code>nn.GELU</code>, <code>nn.Mish</code> och <code>nn.SiLU</code>

Se boken kapitel 11 för detaljer och hur man implementerar de andra.

Swish och ReLU i praktiken vanligast, men tanh är viktig för LLMer.

## Normalisering